# 0. Import

In [265]:
import numpy as np
import sys
import os
import time
import re
from collections import Counter, OrderedDict
from tqdm import tqdm

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import tensorflow as tf

import nltk
nltk.download('stopwords')
nltk.download('punkt')

import torchtext
from torchtext.data import get_tokenizer
from torchtext.vocab import vocab
from torchtext.data.functional import to_map_style_dataset

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
from torch.optim.lr_scheduler import StepLR

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 1. Data preparetion

In [266]:
# Read dataset
path = '/content/dataset.txt'
with open(path, "r") as f:
  text = f.read().lower()

In [267]:
text = re.sub("\n", " EOS SOS ", text)
text = re.sub("\.", "", text)
text = "SOS "+text+" EOS"

In [268]:
text[:100]

"SOS i can't say how every time i ever put my arms around you i felt that i was home EOS SOS since i "

In [269]:
# with open("dataset.txt", "w") as f:
#   f.write(text)

In [270]:
# Tokenize words and create a vocabulary
tokens = word_tokenize(text)
counter = Counter(tokens)
sorted_by_freq_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_by_freq_tuples)
vocabulary = vocab(ordered_dict)
unk_token = '<unk>'
vocabulary.insert_token(unk_token, 0)
vocabulary.set_default_index(0)

In [271]:
print("Total number of words", len(tokens))
print("Number of unique words", len(vocabulary))

Total number of words 10763
Number of unique words 1284


In [272]:
# Create embeddings
# Reference: "https://towardsdatascience.com/multiclass-text-classification-using-lstm-in-pytorch-eac56baed8df"

glove_path = '/content/drive/MyDrive/Study/ML/glove.6B.100d.txt'

# read GloVe data
def load_glove_vectors(glove_file):
    word_vectors = {}
    with open(glove_file) as f:
        for line in f:
            split = line.split()
            word_vectors[split[0]] = np.array([float(x) for x in split[1:]])
    return word_vectors

# create embedding matrix
def get_emb_matrix(pretrained, vocab, emb_size = 100):
    vocab_size = len(vocab)
    W = np.zeros((vocab_size, emb_size), dtype="float32")
    for word in vocab.get_itos():
        if word in pretrained:
            W[vocab[word]] = pretrained[word]
        else:
            W[vocab[word]] = np.random.uniform(-0.25,0.25, emb_size)  
    return W

word_vecs = load_glove_vectors(glove_path)
pretrained_weights = get_emb_matrix(word_vecs, vocabulary)

# 2. Create data loaders

In [273]:
WORD_FOR_PREDICT = 2

In [274]:
def create_dataset(text_list, pred_words=3):
    labels, texts = [], []
    stop_token = vocabulary["EOS"]
    for i in range(len(text_list)-pred_words):
        word_tokens = []
        for j in range(pred_words+1): 
          word_tokens.append(vocabulary[text_list[i+j]])
        if stop_token in word_tokens[:pred_words]:
          continue
        else:
          texts.append(word_tokens[:pred_words])
          labels.append(word_tokens[-1])
    return TensorDataset(torch.from_numpy(np.array(texts)), torch.from_numpy(np.array(labels)))

In [275]:
train_split = int(len(tokens)*0.85)
test_split = len(tokens)-train_split
train_dataset = create_dataset(tokens[:train_split], WORD_FOR_PREDICT)
test_dataset = create_dataset(tokens[-test_split:], WORD_FOR_PREDICT)
whole_dataset = create_dataset(tokens, WORD_FOR_PREDICT)

In [276]:
BATCH_SIZE = 150
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
whole_loader = DataLoader(whole_dataset, batch_size=BATCH_SIZE, shuffle=True)

# 3. Training functions

In [277]:
def acc(prediction, label):
    pred = torch.argmax(prediction, dim=1)
    return torch.sum(pred == label.squeeze()).item()

In [278]:
def train_step(model, loader, loss_fn, optimizer, scheduler, device):
    ''' Calculate loss and accuracy for one epoch'''
    train_losses = []
    perpls = []
    total_acc, total_count = 0, 0
    for batch_idx, (text, label) in enumerate(loader):

        text = text.to(device)
        label = label.to(device)

        prediction = model(text)

        loss_value = loss_fn(prediction, label)
        loss_value.backward()
        train_losses.append(loss_value.item())
        perpls.append(torch.exp(loss_value).item())
        
        optimizer.step()
        optimizer.zero_grad()
        total_acc += acc(prediction, label)
        total_count += label.size(0)
    #scheduler.step()
    return np.mean(train_losses), total_acc/total_count, np.mean(perpls)

In [279]:
def val_step(model, loader, loss_fn, device):
    ''' Calculate accuracy for one epoch'''
    total_acc, total_count = 0, 0
    perpls = []
    with torch.no_grad():
      y_pred, y_true = [], []
      for batch_idx, (text, label) in enumerate(loader):          
          text = text.to(device) 
          label = label.to(device)
          prediction = model(text)

          loss_value = loss_fn(prediction, label)
          perpls.append(torch.exp(loss_value).item())

          total_acc += acc(prediction, label)
          total_count += label.size(0)
    return total_acc/total_count, np.mean(perpls)

# 4. Create model

In [280]:
class LSTM_Glove(nn.Module):
    ''' LSTM model with GloVe embeddings'''

    def __init__(self, vocab_size, embedding_dim, hidden_dim, no_layers, glove_weights):
        super().__init__()

        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.no_layers = no_layers

        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(glove_weights), padding_idx=0, freeze=False)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, no_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, vocab_size)
        self.dropout = nn.Dropout(0.2)
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x, (ht, ct) = self.lstm(x)
        x = self.dropout(ht[-1])
        x = self.linear(x)
        x = self.softmax(x)
        return x

In [281]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [282]:
# define parameters
VOCAB_SIZE = len(vocabulary)
NO_LAYERS = 1
EMBED_DIM = 100
HIDDEN_DIM = 300
EPOCHS = 100

In [283]:
# initialize a model with trainable embeddings
model_emb = LSTM_Glove(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM, NO_LAYERS, pretrained_weights).to(device)
lr=0.005
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model_emb.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=70, gamma=0.1)

In [284]:
model_emb

LSTM_Glove(
  (embedding): Embedding(1284, 100, padding_idx=0)
  (lstm): LSTM(100, 300, batch_first=True)
  (linear): Linear(in_features=300, out_features=1284, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (softmax): Softmax(dim=None)
)

# 5. Training and evaluation

In [285]:
# train with training data and validate with validation data
EPOCHS = 250
for epoch in tqdm(range(EPOCHS)):
    model_emb.train()
    train_loss, train_acc, p = train_step(model_emb, whole_loader, criterion, optimizer, scheduler, device)
    print(f"Epochs = {epoch}, Training Loss : {train_loss}; Training Accuracy: {train_acc}; Peprplexity: {p}.\n")
    #model_emb.eval()
    #val_acc, p = val_step(model_emb, valid_loader, criterion, device)
    #print(f"Epochs = {epoch}, Validation Accuracy : {val_acc}; Peprplexity: {p}\n")
    #scheduler.step()

  1%|          | 2/250 [00:00<00:43,  5.64it/s]

Epochs = 0, Training Loss : 7.08818176814488; Training Accuracy: 0.07205679771113702; Peprplexity: 1198.0077795603918.

Epochs = 1, Training Loss : 7.085540748777843; Training Accuracy: 0.07354032001695454; Peprplexity: 1194.8969067770338.



  2%|▏         | 4/250 [00:00<00:42,  5.80it/s]

Epochs = 2, Training Loss : 7.085320639231848; Training Accuracy: 0.07375225177492847; Peprplexity: 1194.5829864986358.

Epochs = 3, Training Loss : 7.085349907950749; Training Accuracy: 0.07375225177492847; Peprplexity: 1194.6504642547122.



  2%|▏         | 6/250 [00:01<00:41,  5.87it/s]

...

Epochs = 244, Training Loss : 6.647709816221207; Training Accuracy: 0.5123450249019815; Peprplexity: 771.6923053075396.

Epochs = 245, Training Loss : 6.646436842661055; Training Accuracy: 0.5132987178128643; Peprplexity: 770.6211131262401.



 99%|█████████▉| 248/250 [00:44<00:00,  5.64it/s]

Epochs = 246, Training Loss : 6.6464888784620495; Training Accuracy: 0.5131927519338773; Peprplexity: 770.7349882579986.

Epochs = 247, Training Loss : 6.644553706759498; Training Accuracy: 0.5151001377556427; Peprplexity: 769.3047601609003.



100%|██████████| 250/250 [00:44<00:00,  5.61it/s]

Epochs = 248, Training Loss : 6.644396910591731; Training Accuracy: 0.5158418989085515; Peprplexity: 769.2342141772074.

Epochs = 249, Training Loss : 6.644499884711371; Training Accuracy: 0.5154180353926036; Peprplexity: 769.0108148484003.



In [286]:
def continue_seq(tokens, pred):
    w = []
    for i in range(pred, 0, -1):
        w.append(vocabulary[tokens[-pred+1]])
    t = torch.IntTensor([w]).to(device)
    pred = model_emb.forward(torch.reshape(t, (1, pred)))
    
    #a = torch.argmax(pred, dim=1)
    #rand_idx = 0
    a = torch.topk(pred.flatten(), 2).indices
    rand_idx = torch.randint(low=0, high=2, size=(1, 1))[0][0].item()
    new_word = vocabulary.lookup_token(a[rand_idx].item())
    return new_word


In [287]:
starts = text.split("EOS")

In [288]:
def beautify(word_list):
  word_list = word_list[1:]
  word_list[-1] = "."
  text = re.sub(r'\s+([?.,!"])', r'\1', " ".join(word_list))
  return text.capitalize()


In [290]:
for count in range(20):
  i = np.random.randint(0, len(starts))
  start = starts[i].split()[:WORD_FOR_PREDICT]
  while start[-1] != 'EOS':
    start.append(continue_seq(start, WORD_FOR_PREDICT))
  if len(start) <= 5 or len(start) > 25:
    continue
  print(f'{count+1}. {beautify(start)}')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


1. So much i am must have you are great.
2. We to you are the world.
5. You are when i so this hard.
7. As my heart, i am so, you know i am so, and i am by my joy me.
8. You know i am by, i to hold you are the most thing i am so for the world.
10. You are so much i am by, i am by, i if anything.
11. Love, i am so difficult to you know i am by my heart.
12. But 's the world.
14. I am so much.
16. Never saw you know i so much.
17. You know that i am so, i to you are a part of nights other.
20. I and special in world.


# Epochs experiment

RSMprop  
HID_DIM = 110  
LR = 0.005  
Train on the whole dataset.  

50 epochs, peprlexity = 920
* What i feel can that not my not life have, the i best can that not you the, way i.
* You have touched to am be you that.
* Your name is for that my i life have, to my be, the i way can that be i.
* Ever since you the up most life,, i i can can not with have i that of the life best, of i you have, a most, i i can can to to you you life up, a i of can my be, with i you have up the the way most.
* I want to and you me.
* You have the the most of i my have you that up you.
* You continue to with you and.
* Your positivity is that to i be can a not with the i of you my.
* That thing you that.
* You have such the that way i that have my that life i with have you to.

100 epochs, perplexity = 869
* I get down to and you that are i and can that to i you have.
* You are better.
* When i am be with the me way for.
* Looking back to and you i are have.
* Everything would be be a.
* The day you in are my.
* Your hair looks up that.
* You know you that are i the can way be.
* You are everything.
* You bring out to.

150 epochs, perplexity = 859
* I bet you you.
* You are my a for with my and many me in.
* Let your eyes for and you i are to.
* You are simply a and of me my, life i with can me for, me i the have way me you with.
* That thing you that.
* I need you be to.
* How did you i are can the for way me.
* The love you is.
* In the beginning, most 's, you be are a the you most are in.
* I always knew my that and you that are the.

200 epochs, perplexity = 859
* You inspire me to in you a are of.
* You blew my can life be with.
* I am yours in are my a many of.
* Your ability to and you me.
* Your heart must.
* You are a.
* It is not that be me the, most the in way a a with way just.
* Today i have have and me i, have be a a of with my and for i my have life the in way my you and.
* You always make, me the, most my in and the that way you as are my a can you for are me.
* Though we were to my be and in i my can life not and the i most can, have can me for are me the, way i as have you a.



# Optimizer experiment

With RSMprop

Epochs = 219, Training Loss : 6.870341967387372; Training Accuracy: 0.3340213695968917; Peprplexity: 964.6982421875.

Epochs = 219, Validation Accuracy : 0.12114656571119524; Peprplexity: 1192.6570595189144

Sentences:
The feelings, have for the of a day and the world.

* You've got you when of the of my heart and i i have for me of the day and you smile and i i have for my love, have the world, i used in the day world and you.
* Having met you smile and i i used to of a of my heart.
* And i i i have the world, and i have for the of a of my love and the world and you.
* I send of a world.
* My happiness than i i have the of a of the of a of a of a world.
* Who raised, have to of my heart in the world, i have the day and you.
* Gazing into day and i have for the day and i have the of a day world and i used to you smile than i used in my love and you.
* Your kind.
* If my heart and the day.

With Adam

Epochs = 219, Training Loss : 6.9535920016736865; Training Accuracy: 0.251699854298203; Peprplexity: 1048.0043783532567.

Epochs = 219, Validation Accuracy : 0.11898323418063818; Peprplexity: 1194.3838597347863

* I am thank and me, me, and in love in the me that you, you my.
* Dearest — the me that me, you, me that me that you that you that the me my love and the in love and in me my of me my.
* Your name implies for the you, you, and in love and the you, and in me, and me my love and me my love you my love in the you, and the you that me that the you, you that you that you that me that you my.
* When we of you, you my of the me that me my.
* Thank you my of you my of you that me that me, me my of me my of you, and me that you, me that you, you my of you, and me, you my.
* Through this in love and in love and in the me that you that you that the in me, you, me, and in the in the you that the in the in love in me, you, you that me my love you that the you, you my of me that you, me, me my love and in the me, and the in the in the in the in love and the in the me that me my of you that you my love in me that me my love and in love you, me, and in me, me my of me that the in love you that the me, and the me that the me, you my of you, and the in me my love and me that me that me, me, you, you, you, and in me that me my of the me my.
* You have for you that you that you my love in the you that me that the you that the in love you my love you that you that me that me, and the in me that the in love you, and in love in love you, me that you my love you, and the me my.
* Thank you, me, and in me, me my.
* If i not thank and in love you, you, you my.
* Being with in love you, and in love in the you my.